Trying using only 2 models: SLIM_Elastic + IBCF (alpha = 0.12)

Add then 0.685907*that and 1-0.685907 easer

In [1]:
import time
import pandas as pd
import numpy as np
import scipy.sparse as sps
import random as rnd

from scipy.sparse import *

In [2]:
urm_path = './content/data_train.csv'
urm_all_df = pd.read_csv(filepath_or_buffer=urm_path,
                                sep=",",
                                header=0,
                                dtype={0:int, 1:int, 2:float},
                                engine='python')

urm_all_df.columns = ["UserID", "ItemID", "Interaction"]
print ("The number of interactions is {}".format(len(urm_all_df)))

The number of interactions is 478730


In [3]:
URM_all = sps.coo_matrix((urm_all_df["Interaction"].values,
                          (urm_all_df["UserID"].values, urm_all_df["ItemID"].values)))

URM_all

<13025x22348 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in COOrdinate format>

In [4]:
from scipy.sparse import load_npz

In [5]:
S_slim_elastic = load_npz("./content/item_item_similarity/slim_elastic_complete.npz")

In [6]:
S_slim_elastic

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 2197697 stored elements in Compressed Sparse Row format>

In [7]:
S_easer = load_npz("./content/item_item_similarity/easer_complete.npz")

In [8]:
S_easer

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 5044394 stored elements in Compressed Sparse Row format>

In [9]:
S_IBCF = load_npz("./content/item_item_similarity/IBCF_complete.npz")

In [10]:
S_IBCF

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 310959 stored elements in Compressed Sparse Row format>

In [11]:
S_rp3beta = load_npz("./content/item_item_similarity/rp3beta_complete.npz")

In [12]:
S_rp3beta

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 3619038 stored elements in Compressed Sparse Row format>

In [13]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

In [14]:
alpha = 0.12
new_similarity = (1 - alpha) * S_slim_elastic + alpha * S_IBCF
recommender_object = ItemKNNCustomSimilarityRecommender(URM_all)
recommender_object.fit(new_similarity)

ItemKNNCustomSimilarityRecommender: URM Detected 387 ( 3.0%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 126 ( 0.6%) items with no interactions.


In [15]:
from Recommenders.NonPersonalizedRecommender import TopPop
topPopRecommender = TopPop(URM_all)
topPopRecommender.fit()

TopPopRecommender: URM Detected 387 ( 3.0%) users with no interactions.
TopPopRecommender: URM Detected 126 ( 0.6%) items with no interactions.


In [17]:
beta = 0.685907
alpha = 0.12
new_similarity = ((1 - alpha) * S_slim_elastic + alpha * S_IBCF)*beta + (1-beta)*S_easer
recommender_object = ItemKNNCustomSimilarityRecommender(URM_all)
recommender_object.fit(new_similarity)

ItemKNNCustomSimilarityRecommender: URM Detected 387 ( 3.0%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 126 ( 0.6%) items with no interactions.


In [18]:
class Predictor(object):

    def __init__(self, URM, model):
        self.URM = URM
        self.model = model

    def recommend(self, user_id, at=10, exclude_seen=True, users_not_in_train=[]):
        # Check if user_id is a valid index
        if user_id < 0 or user_id >= self.URM.shape[0]:
            print(f"Invalid user_id: {user_id}")
            return

        # Check if user_id not in train use the topRec
        if user_id in users_not_in_train:
            scores = topPopRecommender._compute_item_score([user_id])[0]

        else:# compute the scores using the fitted model
            scores = self.model._compute_item_score(user_id)[0]

        if exclude_seen:
            scores = self.filter_seen(user_id, scores)

        # rank items
        ranking = scores.argsort()[::-1]

        return ranking[:at]


    def filter_seen(self, user_id, scores):

        start_pos = self.URM.indptr[user_id]
        end_pos = self.URM.indptr[user_id+1]

        user_profile = self.URM.indices[start_pos:end_pos]

        scores[user_profile] = -np.inf

        return scores

# Predictions

In [19]:
urm_pred_path = './content/data_target_users_test.csv'

urm_pred_df = pd.read_csv(filepath_or_buffer=urm_pred_path,
                                sep=",",
                                header=0,
                                dtype={0:int},
                                engine='python')

urm_pred_df.columns = ["UserID"]
len(urm_pred_df['UserID'])
print('Unique user id to predict:', urm_pred_df['UserID'].nunique())

Unique user id to predict: 10882


In [20]:
users_not_in_train = urm_pred_df[~urm_pred_df['UserID'].isin(urm_all_df['UserID'])]

print("Users in urm_pred_df but not in urm_all_orgdf:")
print(users_not_in_train)
print(len(users_not_in_train))

users_not_in_train = users_not_in_train['UserID'].to_numpy()

Users in urm_pred_df but not in urm_all_orgdf:
       UserID
54         60
58         65
147       168
223       261
272       316
...       ...
10682   12775
10699   12798
10729   12837
10802   12921
10856   12992

[221 rows x 1 columns]
221


In [21]:
URM_all = URM_all.tocsr()

In [22]:
recommender = Predictor(URM=URM_all, model = recommender_object)

In [23]:
user_id = 61
print(f'Predicting for user - {user_id}')
prediction = recommender.recommend(user_id,users_not_in_train = users_not_in_train)
print(f"The prediction is {prediction}")

Predicting for user - 61
The prediction is [ 192 1076  393 3260 3580 6887 9336 7978  487 1710]


In [24]:
user_id = 60
print(f'Predicting for user - {user_id}')
prediction = recommender.recommend(user_id,users_not_in_train = users_not_in_train)
print(f"The prediction is {prediction}")

Predicting for user - 60
The prediction is [ 2  4  1  7  3  6  8  9 15 20]


In [25]:
pred_df = pd.DataFrame(columns = ['user_id','item_list'])

for userid in urm_pred_df['UserID']:
  recommendations = recommender.recommend(userid, at=10,exclude_seen = True, users_not_in_train=users_not_in_train)
  recommendations = " ".join(str(item) for item in recommendations)
  pred_df.loc[len(pred_df)] = [userid,recommendations]

In [26]:
pred_df

,user_id,item_list
0,1,101 36 123 506 515 403 694 1546 52 254
1,2,1095 47 12 50 1522 28 656 949 11 196
2,3,59 857 4252 2172 536 4 648 956 259 584
3,4,249 28 50 136 139 314 171 146 145 254
4,5,1570 170 77 95 5138 1511 131 116 471 148
...,...,...
10877,13020,6450 6198 6452 7395 6749 7394 4323 105 161 1191
10878,13021,6179 6451 7027 6426 6720 7395 6749 133 13621 1...
10879,13022,1668 1411 809 1446 1674 4688 10789 1013 1258 776
10880,13023,1124 329 706 1107 1290 1534 138 96 208 1532


In [27]:
pred_df.to_csv('./content/predHybridfinal.csv',index=False)